In [ ]:
# Imports and configuration
import os, sqlite3, warnings
from datetime import timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # type: ignore
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Paths (change if needed)
CSV_PATH = r"C:\Users\hp\OneDrive\Desktop\Weather Prediction Model\weather_data.csv"   # <- change this if your CSV is elsewhere
OUTDIR = "/mnt/data"
DB_PATH = os.path.join(OUTDIR, "weather_weather.db")
PRED_CSV_PATH = os.path.join(OUTDIR, "weather_predictions.csv")

print('CSV_PATH =', CSV_PATH)
print('Outputs will be saved to:', OUTDIR)

In [ ]:
# Load CSV and basic checks
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f'CSV not found at {CSV_PATH}. Upload your CSV or change CSV_PATH.')

df = pd.read_csv(CSV_PATH)
print('Loaded CSV with shape:', df.shape)
display(df.head())

# helper to detect columns
def find_date_column(df):
    for col in df.columns:
        if col.lower() in ('date','datetime','day','timestamp'):
            return col
    for col in df.columns:
        try:
            pd.to_datetime(df[col])
            return col
        except Exception:
            continue
    return None

def find_temp_and_rain_columns(df):
    temp_col = None; rain_col = None
    for col in df.columns:
        lname = col.lower()
        if temp_col is None and ('temp' in lname or 'temperature' in lname):
            temp_col = col
        if rain_col is None and ('rain' in lname or 'precip' in lname):
            rain_col = col
    return temp_col, rain_col

date_col = find_date_column(df)
if date_col is not None:
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df = df.sort_values(by=date_col).reset_index(drop=True)
else:
    df = df.reset_index().rename(columns={'index':'synthetic_index'})
    date_col = 'synthetic_index'
    df[date_col] = pd.to_datetime(df[date_col], unit='D', origin='1970-01-01')

temp_col, rain_col = find_temp_and_rain_columns(df)
if temp_col is None or rain_col is None:
    raise ValueError('Could not detect temp or rain columns. Ensure names include "temp" and "rain"/"precip".\nFound columns: ' + ', '.join(df.columns))

df[temp_col] = pd.to_numeric(df[temp_col], errors='coerce')
df[rain_col] = pd.to_numeric(df[rain_col], errors='coerce')
df = df.dropna(subset=[temp_col, rain_col], how='all').reset_index(drop=True)

print('Using date_col =', date_col, ', temp_col =', temp_col, ', rain_col =', rain_col)
print('\nLast 5 rows:')
display(df.tail())

# Save raw table to SQLite DB (replace)
os.makedirs(OUTDIR, exist_ok=True)
conn = sqlite3.connect(DB_PATH)
df_for_db = df.copy()
if 'predicted' not in df_for_db.columns:
    df_for_db['predicted'] = 0
df_for_db.to_sql('weather', conn, if_exists='replace', index=False)
conn.commit(); conn.close()
print('Saved raw data to DB at', DB_PATH)

In [ ]:
# Pattern analysis (last 7 days) and feature engineering
last7 = df.tail(7).copy()
print('Pattern analysis (last 7 days):')
temps = last7[temp_col].values
rains = last7[rain_col].values
print(' - temp_mean:', float(np.nanmean(temps)))
print(' - temp_median:', float(np.nanmedian(temps)))
print(' - temp_std:', float(np.nanstd(temps)))
print(' - temp_trend:', 'increasing' if temps[-1] > temps[0] else ('decreasing' if temps[-1] < temps[0] else 'flat'))
print(' - rain_total:', float(np.nansum(rains)))
print(' - rain_days:', int(np.sum(~np.isnan(rains) & (rains>0))))

# plots for last 7 days
plt.figure(figsize=(8,3)); plt.plot(last7[date_col], last7[temp_col], marker='o'); plt.title('Temperature - last 7 days'); plt.xlabel('Date'); plt.ylabel('Temperature'); plt.tight_layout(); plt.show()
plt.figure(figsize=(8,3)); plt.plot(last7[date_col], last7[rain_col], marker='o'); plt.title('Rainfall - last 7 days'); plt.xlabel('Date'); plt.ylabel('Rainfall'); plt.tight_layout(); plt.show()

# Feature engineering: lags and rolling features
nlags = 3
df_feat = df[[date_col, temp_col, rain_col]].copy().set_index(date_col)
for lag in range(1, nlags+1):
    df_feat[f'temp_lag_{lag}'] = df_feat[temp_col].shift(lag)
    df_feat[f'rain_lag_{lag}'] = df_feat[rain_col].shift(lag)
df_feat['temp_roll_3'] = df_feat[temp_col].rolling(window=3, min_periods=1).mean().shift(1)
df_feat['rain_roll_3'] = df_feat[rain_col].rolling(window=3, min_periods=1).mean().shift(1)

df_clean = df_feat.dropna().copy()
print('\nAfter feature creation, rows available for modelling:', len(df_clean))
display(df_clean.head())

In [ ]:
# Modeling and evaluation (Temperature and Rainfall separately)
feature_cols = [c for c in df_clean.columns if c not in (temp_col, rain_col)]
X = df_clean[feature_cols].values
y_temp = df_clean[temp_col].values
y_rain = df_clean[rain_col].values

def evaluate_models(X, y, n_splits=3):
    tscv = TimeSeriesSplit(n_splits=max(1, min(n_splits, len(X)-1)))
    models = {
        'LinearRegression': Pipeline([('lr', LinearRegression())]),
        'DecisionTree': Pipeline([('dt', DecisionTreeRegressor(random_state=42))]),
        'SVR': Pipeline([('scaler', StandardScaler()), ('svr', SVR())])
    }
    results = {}
    for name, model in models.items():
        maes, rmses, r2s = [], [], []
        splits = list(tscv.split(X)) if len(X) > 1 else []
        if len(splits) == 0:
            if len(X) < 2:
                maes.append(np.nan); rmses.append(np.nan); r2s.append(np.nan)
            else:
                train_idx = np.arange(max(1, len(X)-1)); test_idx = np.array([len(X)-1]); splits = [(train_idx, test_idx)]
        for train_idx, test_idx in splits:
            Xtr, Xte = X[train_idx], X[test_idx]
            ytr, yte = y[train_idx], y[test_idx]
            try:
                model.fit(Xtr, ytr)
                ypred = model.predict(Xte)
                maes.append(mean_absolute_error(yte, ypred))
                rmses.append(mean_squared_error(yte, ypred, squared=False))
                r2s.append(r2_score(yte, ypred) if len(yte)>1 else np.nan)
            except Exception as e:
                maes.append(np.nan); rmses.append(np.nan); r2s.append(np.nan)
        results[name] = {'mae_mean': float(np.nanmean(maes)), 'rmse_mean': float(np.nanmean(rmses)), 'r2_mean': float(np.nanmean(r2s)), 'model': model}
    return results

print('Training & evaluating models for Temperature...')
temp_results = evaluate_models(X, y_temp, n_splits=3)
for name, res in temp_results.items():
    print(f" - {name}: MAE={res['mae_mean']:.4f}, RMSE={res['rmse_mean']:.4f}, R2={res['r2_mean']:.4f}")

print('\nTraining & evaluating models for Rainfall...')
rain_results = evaluate_models(X, y_rain, n_splits=3)
for name, res in rain_results.items():
    print(f" - {name}: MAE={res['mae_mean']:.4f}, RMSE={res['rmse_mean']:.4f}, R2={res['r2_mean']:.4f}")

# Choose best by RMSE
best_temp_name = min(temp_results.keys(), key=lambda k: temp_results[k]['rmse_mean'] if not np.isnan(temp_results[k]['rmse_mean']) else 1e9)
best_rain_name = min(rain_results.keys(), key=lambda k: rain_results[k]['rmse_mean'] if not np.isnan(rain_results[k]['rmse_mean']) else 1e9)
best_temp_model = temp_results[best_temp_name]['model']
best_rain_model = rain_results[best_rain_name]['model']

# Fit on entire cleaned data
best_temp_model.fit(X, y_temp)
best_rain_model.fit(X, y_rain)
print('\nSelected models -> Temperature:', best_temp_name, ', Rainfall:', best_rain_name)

In [ ]:
# Next-day (8th day) prediction and save outputs
# Prepare next-day features from most recent values
last_index = df_feat.index.max()
next_features = {}
for lag in range(1, nlags+1):
    next_features[f'temp_lag_{lag}'] = df_feat[temp_col].iloc[-lag] if len(df_feat) >= lag else np.nan
    next_features[f'rain_lag_{lag}'] = df_feat[rain_col].iloc[-lag] if len(df_feat) >= lag else np.nan
next_features['temp_roll_3'] = df_feat[temp_col].tail(3).mean() if len(df_feat) >= 1 else np.nan
next_features['rain_roll_3'] = df_feat[rain_col].tail(3).mean() if len(df_feat) >= 1 else np.nan

X_next = np.array([next_features[c] for c in feature_cols]).reshape(1, -1)
temp_pred = best_temp_model.predict(X_next)[0]
rain_pred = best_rain_model.predict(X_next)[0]

# compute predicted date (assume daily)
try:
    if isinstance(last_index, pd.Timestamp):
        if len(df_feat.index) >= 2:
            delta = df_feat.index[-1] - df_feat.index[-2]
            next_date = df_feat.index[-1] + delta
        else:
            next_date = df_feat.index[-1] + timedelta(days=1)
    else:
        next_date = pd.to_datetime(df_feat.index.max()) + timedelta(days=1)
except Exception:
    next_date = pd.to_datetime(df_feat.index.max()) + timedelta(days=1)

pred_row = {date_col: next_date, temp_col: float(temp_pred), rain_col: float(rain_pred)}
pred_row['predicted'] = 1
pred_row['model_temp'] = best_temp_name
pred_row['model_rain'] = best_rain_name
pred_row.update(next_features)

pred_df = pd.DataFrame([pred_row])
print('\nNext-day prediction (8th day):')
display(pred_df.rename(columns={date_col: 'date', temp_col: 'temperature', rain_col: 'rainfall'}))

# Append prediction to DB
conn = sqlite3.connect(DB_PATH)
df_for_db = pd.read_sql_query('SELECT * FROM weather LIMIT 1;', conn)  # just to get schema
pred_to_write = pred_df.copy()
# ensure all columns exist
for col in df_for_db.columns:
    if col not in pred_to_write.columns:
        pred_to_write[col] = np.nan
pred_to_write = pred_to_write[df_for_db.columns]
pred_to_write.to_sql('weather', conn, if_exists='append', index=False)
conn.commit(); conn.close()
print('Appended prediction to DB at', DB_PATH)

# Save prediction CSV
out_pred = pred_df.rename(columns={date_col: 'date', temp_col: 'temperature', rain_col: 'rainfall'})
out_pred.to_csv(PRED_CSV_PATH, index=False)
print('Saved prediction CSV to', PRED_CSV_PATH)

# Plots: actual vs predicted for last part
try:
    y_temp_all = best_temp_model.predict(X)
    y_rain_all = best_rain_model.predict(X)
    nplot = min(30, len(df_clean))
    idx = df_clean.index[-nplot:]
    plt.figure(figsize=(10,3))
    plt.plot(idx, df_clean[temp_col].iloc[-nplot:], label='actual')
    plt.plot(idx, y_temp_all[-nplot:], linestyle='--', label='predicted')
    plt.title('Temperature: Actual vs Predicted (last points)'); plt.legend(); plt.tight_layout(); plt.show()

    plt.figure(figsize=(10,3))
    plt.plot(idx, df_clean[rain_col].iloc[-nplot:], label='actual')
    plt.plot(idx, y_rain_all[-nplot:], linestyle='--', label='predicted')
    plt.title('Rainfall: Actual vs Predicted (last points)'); plt.legend(); plt.tight_layout(); plt.show()
except Exception as e:
    print('Plotting failed:', e)

print('\nNotebook run complete. Outputs:') 
print(' - Prediction CSV:', PRED_CSV_PATH)
print(' - SQLite DB:', DB_PATH)

In [ ]:
# Cell 1: Import additional libraries for EDA
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

print("=== EXPLORATORY DATA ANALYSIS ===")

In [ ]:
# Cell 2: Basic data overview and statistics
print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nBasic Info:")
print(df.info())
print("\nDescriptive Statistics:")
display(df.describe())
print("\nMissing Values:")
print(df.isnull().sum())

In [ ]:
# Cell 3: Rainfall distribution analysis
plt.figure(figsize=(15, 5))

# Pie chart for rainfall distribution
plt.subplot(1, 3, 1)
if rain_col in df.columns:
    rainfall_counts = df[rain_col].value_counts()
    # Convert to binary (rain/no rain) if continuous
    if rainfall_counts.shape[0] > 10:  # If continuous, create binary
        df_rain_binary = (df[rain_col] > 0).astype(int)
        rainfall_binary_counts = df_rain_binary.value_counts()
        plt.pie(rainfall_binary_counts.values, 
                labels=['No Rain', 'Rain'], 
                autopct='%1.1f%%', 
                colors=['skyblue', 'lightcoral'])
        plt.title('Rainfall Distribution (Binary)')
    else:
        plt.pie(rainfall_counts.values, 
                labels=rainfall_counts.index, 
                autopct='%1.1f%%')
        plt.title('Rainfall Distribution')
else:
    plt.text(0.5, 0.5, 'Rain column not found', ha='center', va='center')
    plt.title('Rainfall Distribution - Data Not Available')

# Rainfall vs temperature relationship
plt.subplot(1, 3, 2)
if rain_col in df.columns and temp_col in df.columns:
    # Create binary rain for scatter plot
    df_eda = df.copy()
    df_eda['rain_binary'] = (df_eda[rain_col] > 0).astype(int)
    sns.scatterplot(data=df_eda, x=temp_col, y=rain_col, hue='rain_binary', alpha=0.6)
    plt.title('Temperature vs Rainfall')
else:
    plt.text(0.5, 0.5, 'Required columns not found', ha='center', va='center')
    plt.title('Temperature vs Rainfall')

# Time series of rainfall
plt.subplot(1, 3, 3)
if rain_col in df.columns and date_col in df.columns:
    plt.plot(df[date_col], df[rain_col], alpha=0.7)
    plt.title('Rainfall Over Time')
    plt.xticks(rotation=45)
else:
    plt.text(0.5, 0.5, 'Required columns not found', ha='center', va='center')
    plt.title('Rainfall Over Time')

plt.tight_layout()
plt.show()

In [ ]:
# Cell 4: Grouped analysis by rainfall
print("Grouped Analysis by Rainfall (Binary):")

# Create binary rainfall column for analysis
df_eda = df.copy()
if rain_col in df.columns:
    df_eda['rain_binary'] = (df_eda[rain_col] > 0).astype(int)
    
    # Display grouped statistics
    numeric_cols = df_eda.select_dtypes(include=[np.number]).columns
    numeric_cols = [col for col in numeric_cols if col != 'rain_binary']
    
    if len(numeric_cols) > 0:
        grouped_stats = df_eda.groupby('rain_binary')[numeric_cols].mean()
        print("\nMean values by rainfall status:")
        display(grouped_stats)
        
        # Key observations
        print("\nKey Observations:")
        if temp_col in numeric_cols:
            temp_diff = grouped_stats[temp_col].diff().iloc[-1]
            print(f"- Temperature difference (Rain vs No Rain): {temp_diff:.2f}")
            
        # Check for humidity column
        humidity_cols = [col for col in df.columns if 'humid' in col.lower()]
        if humidity_cols and humidity_cols[0] in numeric_cols:
            humid_col = humidity_cols[0]
            humid_diff = grouped_stats[humid_col].diff().iloc[-1]
            print(f"- Humidity difference (Rain vs No Rain): {humid_diff:.2f}")
            
        # Check for cloud column
        cloud_cols = [col for col in df.columns if 'cloud' in col.lower()]
        if cloud_cols and cloud_cols[0] in numeric_cols:
            cloud_col = cloud_cols[0]
            cloud_diff = grouped_stats[cloud_col].diff().iloc[-1]
            print(f"- Cloud cover difference (Rain vs No Rain): {cloud_diff:.2f}")
            
        # Check for wind speed
        wind_cols = [col for col in df.columns if 'wind' in col.lower() and 'speed' in col.lower()]
        if wind_cols and wind_cols[0] in numeric_cols:
            wind_col = wind_cols[0]
            wind_diff = grouped_stats[wind_col].diff().iloc[-1]
            print(f"- Wind speed difference (Rain vs No Rain): {wind_diff:.2f}")
else:
    print("Rain column not available for grouped analysis")

In [ ]:
# Cell 5: Distribution plots for all numeric features
print("Distribution of Numeric Features:")

# Get numeric columns (excluding date and target columns)
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
# Remove date column if it was converted to numeric
if date_col in numeric_features:
    numeric_features.remove(date_col)
# Remove target columns from features list
if temp_col in numeric_features:
    numeric_features.remove(temp_col)
if rain_col in numeric_features:
    numeric_features.remove(rain_col)

print(f"Numeric features to analyze: {numeric_features}")

if numeric_features:
    # Calculate grid dimensions
    n_features = len(numeric_features)
    n_cols = 4
    n_rows = (n_features + n_cols - 1) // n_cols
    
    plt.figure(figsize=(15, 4*n_rows))
    
    for i, feature in enumerate(numeric_features, 1):
        plt.subplot(n_rows, n_cols, i)
        sns.histplot(df[feature], kde=True, bins=30)
        plt.title(f'Distribution of {feature}')
        plt.xlabel(feature)
        plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
else:
    print("No numeric features found for distribution analysis")

In [ ]:
# Cell 6: Box plots for outlier detection
print("Box Plots for Outlier Detection:")

if numeric_features:
    plt.figure(figsize=(15, 4*n_rows))
    
    for i, feature in enumerate(numeric_features, 1):
        plt.subplot(n_rows, n_cols, i)
        sns.boxplot(y=df[feature])
        plt.title(f'Box Plot of {feature}')
        plt.ylabel(feature)
    
    plt.tight_layout()
    plt.show()
    
    # Outlier analysis
    print("\nOutlier Analysis (using IQR method):")
    for feature in numeric_features:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
        print(f"{feature}: {len(outliers)} outliers ({len(outliers)/len(df)*100:.1f}%)")
else:
    print("No numeric features found for box plot analysis")

In [ ]:
# Cell 7: Correlation analysis
print("Correlation Analysis:")

# Prepare data for correlation (include all numeric columns)
corr_columns = [temp_col, rain_col] + numeric_features
corr_columns = [col for col in corr_columns if col in df.columns]

if len(corr_columns) > 1:
    # Calculate correlation matrix
    correlation_matrix = df[corr_columns].corr()
    
    # Plot heatmap
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
    sns.heatmap(correlation_matrix, 
                mask=mask,
                annot=True, 
                cmap='coolwarm', 
                center=0,
                square=True,
                fmt='.2f',
                cbar_kws={'shrink': 0.8})
    plt.title('Feature Correlation Heatmap')
    plt.tight_layout()
    plt.show()
    
    # Identify highly correlated features
    print("\nHighly Correlated Features (|r| > 0.8):")
    high_corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.8:
                high_corr_pairs.append((
                    correlation_matrix.columns[i],
                    correlation_matrix.columns[j],
                    correlation_matrix.iloc[i, j]
                ))
    
    if high_corr_pairs:
        for feature1, feature2, corr_value in high_corr_pairs:
            print(f"- {feature1} vs {feature2}: {corr_value:.3f}")
    else:
        print("No highly correlated feature pairs found (|r| > 0.8)")
else:
    print("Not enough numeric columns for correlation analysis")